<a href="https://colab.research.google.com/github/inepicone/product_footprint/blob/main/mvp_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

✅ Simula el escaneo de un producto al azar desde Open Food Facts.

✅ Pide la ubicación del usuario manualmente (puede ser una ciudad o coordenadas).

✅ Obtiene la ubicación del país de origen del producto.

✅ Calcula la distancia entre el usuario y el origen del producto.

✅ Calcula el consumo estimado de nafta en el transporte.

✅ Devuelve un ticket ecológico con los datos.



In [ ]:
!pip install geopy requests



In [ ]:
import requests
import random
from geopy.geocoders import Nominatim
from geopy.distance import geodesic


In [ ]:

# Función para obtener un producto al azar desde Open Food Facts
def obtener_producto():
    url = "https://world.openfoodfacts.org/api/v0/product/737628064502.json"  # Cambia el código de barras si quieres otro
    response = requests.get(url)
    data = response.json()

    if "product" in data:
        producto = data["product"]
        nombre = producto.get("product_name", "Desconocido")
        pais_origen = producto.get("origins", "Desconocido")
        return nombre, pais_origen
    else:
        return None, None

# Función para obtener coordenadas de una ciudad
def obtener_coordenadas(ubicacion):
    geolocator = Nominatim(user_agent="my-geocoding-app")  # Replace with your descriptive app name

    # Implement retry mechanism
    for i in range(3):  # Try 3 times
        try:
            location = geolocator.geocode(ubicacion)
            if location:
                return (location.latitude, location.longitude)
            else:
                return None
        except geopy.exc.GeocoderServiceError as e:
            if e.code == 403:  # If rate limited (HTTP 403), wait and retry
                print(f"Rate limited, retrying in {2**i} seconds...")
                sleep(2**i)  # Exponential backoff
            else:
                raise  # Re-raise other errors

    return None  # Return None if all retries fail
# Función para calcular la distancia y consumo de combustible
def calcular_impacto(ubicacion_usuario, ubicacion_origen):
    coords_usuario = obtener_coordenadas(ubicacion_usuario)
    coords_origen = obtener_coordenadas(ubicacion_origen)

    if coords_usuario and coords_origen:
        distancia_km = geodesic(coords_usuario, coords_origen).kilometers
        consumo_nafta_litros = distancia_km / 3  # Suponiendo 3 km por litro
        return distancia_km, consumo_nafta_litros
    else:
        return None, None



# Nueva sección

In [ ]:
# Simulación
producto, pais_origen = obtener_producto()
ubicacion_usuario = input("Ingresa tu ubicación (Ej: Madrid, España): ")

if producto and pais_origen:
    distancia, nafta = calcular_impacto(ubicacion_usuario, pais_origen)
    if distancia and nafta:
        print("\n--- Ticket Ecológico ---")
        print(f"Producto: {producto}")
        print(f"Origen: {pais_origen}")
        print(f"Distancia recorrida: {distancia:.2f} km")
        print(f"Consumo estimado de nafta: {nafta:.2f} litros")
    else:
        print("No se pudo calcular la distancia.")
else:
    print("No se encontró el producto.")


Ingresa tu ubicación (Ej: Madrid, España): Espoo, Finland

--- Ticket Ecológico ---
Producto: Thai peanut noodle kit includes stir-fry rice noodles & thai peanut seasoning
Origen: Thailand
Distancia recorrida: 7816.65 km
Consumo estimado de nafta: 2605.55 litros
